<h1> Load annotations </h1>

In [ ]:
import os
import sys
import random
import math
import pickle
import re
import time
import numpy as np
import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import keras
import skimage.io

LIB_DIRECTORY = '/root/alok/repos/cv_research/lib/'
sys.path.insert(0, os.path.join(LIB_DIRECTORY, 'maskrcnn'))
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
# import mextra.utils as extra_utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import json
import os
import glob
import sys
LIB_DIRECTORY = '/root/alok/repos/cv_research/lib/'
sys.path.insert(0, LIB_DIRECTORY)

import pandas as pd
from pycocotools.coco import COCO

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs", "body_part_segmentation_20181031_21H02")



In [ ]:
annotation_file = '/root/alok/data/images/annotation_file_test_set.json'

In [ ]:
coco_output = json.load(open(annotation_file))

<h1> Visualize the Results </h1>

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

In [ ]:
coco = COCO(annotation_file)

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
def plot_annotations(image_id):    
    image_data = coco.loadImgs([image_id])[0]
    image_file_path = image_data['local_path']
    annotation_ids = coco.getAnnIds(imgIds=[image_id])
    annotations = coco.loadAnns(annotation_ids)

    # load and display instance annotations
    image = skimage.io.imread(image_file_path)
    f, ax = plt.subplots(1, figsize=(20, 20))
    ax.imshow(image)
    ax.axis('off')
    coco.showAnns(annotations)
    
    # display bounding boxes
    for ann in annotations:
        bbox = ann['bbox']
        rectangle = Rectangle((bbox[1], bbox[0]), bbox[3]-bbox[1], bbox[2]-bbox[0], edgecolor='w', facecolor=None, fill=False, linestyle='--', linewidth=2)
        ax.add_patch(rectangle)
#         category_id = ann['category_id']
        category_id = ann['id']
        ax.text(bbox[1], bbox[0] - 10, category_id, fontsize=16, color='w')


    
    


In [ ]:
plot_annotations(527)

In [ ]:
plot_annotations(2222)

<h1> Get biomass estimates </h1>

In [ ]:
'''
Define some helper functions (most of this code is adapted from cv_algorithms)
'''

import numpy as np
import pandas as pd
import statsmodels.api as sm
from PIL import Image, ImageDraw
from scipy.optimize import linear_sum_assignment
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances


FOCAL_LENGTH = 0.0107
BASELINE = 0.135
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000
COST_THRESHOLD = 100.0  # another magic number

# TODO (@Alok): this is hardcoded - not good, should come directly from coco file
CATEGORIES = [
    {'id': 1, 'name': 'salmon'},
    {'id': 2, 'name': 'Head'},
    {'id': 3, 'name': 'Caudal Fin'},
    {'id': 4, 'name': 'Dorsal Fin'},
    {'id': 5, 'name': 'Adipose Fin'},
    {'id': 6, 'name': 'Anal Fin'},
    {'id': 7, 'name': 'Pelvic Fin'},
    {'id': 8, 'name': 'Pectoral Fin'},
    {'id': 9, 'name': 'Eye'}
]

WHOLE_FISH = 'salmon'

# TODO (@Alok): this is hardcoded - not good, should come from a file
BODY_PARTS_REQUIRED = ['Head', 'Dorsal Fin', 'Anal Fin']

MODEL_PATH = '/root/data/models/biomass/model.pickle'
COMPONENTS_PATH = '/root/data/models/biomass/components.npy'
IQRS_PATH = '/root/data/models/biomass/iqrs.pkl'


def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_HEIGHT / 2.0  
    image_center_y = PIXEL_COUNT_WIDTH / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / 4096)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / 3000)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])


def depth_from_disp(disp):
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth


def get_fish_detections(annotations, categories, whole_fish="salmon"):
    
    """ Get fish detections -- i.e. get all whole fish and body parts corresponding to that whole fish in a single frame """
    
    whole_fish_id = [cat for cat in categories if cat['name'] == whole_fish][0]['id']
    fish = []
    body_parts = []

    # first we separate whole salmon from body parts and give a unique identifier
    for annotation in annotations:
        if annotation['category_id'] == whole_fish_id:
            fish.append(annotation)
        else:
            body_parts.append(annotation)

    # second we match body parts with whole salmon
    # @TODO (Thomas) this is not great because it does not take edges cases into account.
    for part in body_parts:
        bbox = part['bbox']
        part_centroid = [np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]
        for f in fish:
            bbox = f['bbox']
            if bbox[0] < part_centroid[0] < bbox[2] and bbox[1] < part_centroid[1] < bbox[3]:
                part['fish_ann_id'] = f['id']
                break
            
    # @TODO (Alok) this is redudant work here -- merge this with the part above
    fish_detections = []
    for f in fish:
        detection = {
            'fish': f,
            'body_parts': []
        }
        for part in body_parts:
            if part.get('fish_ann_id') == f['id']:
                detection['body_parts'].append(part)
        fish_detections.append(detection)
        
    return fish_detections


def get_stereo_fish_detections(left_fish_detections, right_fish_detections, categories, whole_fish="salmon"):
    
    """match the left and right fish detections to return a list of stereo fish detections """
    
    # get the salmon category id
    whole_fish_id = [cat for cat in categories if cat['name'] == whole_fish][0]['id']

    # let's use x1, x2 to match bboxes
    left_bottom_top_edge_locations = []
    left_ids = []
    for detection in left_fish_detections:
        f = detection['fish']
        assert f['category_id'] == whole_fish_id, 'Annotation should be a whole fish but is a body part'
        bbox = f['bbox']
        bottom_top_edge_location = [bbox[2], bbox[0]]
        left_ids.append(f['id'])
        left_bottom_top_edge_locations.append(bottom_top_edge_location)

    right_bottom_top_edge_locations = []
    right_ids = []
    for detection in right_fish_detections:
        f = detection['fish']
        assert f['category_id'] == whole_fish_id, 'Annotation should be a whole fish but is a body part'
        bbox = f['bbox']
        bottom_top_edge_location = [bbox[2], bbox[0]]
        right_ids.append(f['id'])
        right_bottom_top_edge_locations.append(bottom_top_edge_location)

    # euclidean distance in (x1, x2) space
    cost_matrix = euclidean_distances(left_bottom_top_edge_locations, right_bottom_top_edge_locations)
    
    # hungarian algorithm to minimize weights in bipartite graph
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    stereo_fish_detections = []
    for (r, c) in zip(row_ind, col_ind):
        if cost_matrix[r, c] < COST_THRESHOLD:
            left_fish_detection = [l for l in left_fish_detections if l['fish']['id'] == left_ids[r]][0]
            right_fish_detection = [r for r in right_fish_detections if r['fish']['id'] == right_ids[c]][0]
            
            stereo_fish_detections.append({
                'left': left_fish_detection,
                'right': right_fish_detection
            })
    
    return stereo_fish_detections


def weight_estimator(left_matched_parts, right_matched_parts, required_categories, model, components, iqrs):
    left_category_ids = [ann['category_id'] for ann in left_matched_parts]
    right_category_ids = [ann['category_id'] for ann in right_matched_parts]
    
    required_body_part_ids = sorted([cat['id'] for cat in required_categories])
    
    left_valid = all([body_part in left_category_ids for body_part in required_body_part_ids])
    right_valid = all([body_part in right_category_ids for body_part in required_body_part_ids])
    
    if left_valid and right_valid:
        """take left and right body parts and calculates the weights"""
        
        # create dataset that will contain pairwise distances
        distances = [str(c) + str(k) for (i, c) in enumerate(required_body_part_ids) for k in required_body_part_ids[i + 1:]]
        dataset = {}
        for d in distances:
            dataset[d] = []

        # calculate disparities & world coordinates
        world_coordinates = {}
        for cat in required_categories:
            
            # calculate left centroid   
            left_part = [part for part in left_matched_parts if part['category_id'] == cat['id']][0]
            seg = left_part['segmentation'][0]
            poly = np.array(seg).reshape((int(len(seg) / 2), 2))
            p = [(r[0], r[1]) for r in poly]
            left_mask = Image.new('L', (4096, 3000), 0)
            ImageDraw.Draw(left_mask).polygon(p, outline=1, fill=1)
            left_mask = np.array(left_mask)
            x, y = np.nonzero(left_mask)
            left_centroid = [np.mean(x), np.mean(y)]

            # calculate right centroid
            right_part = [part for part in right_matched_parts if part['category_id'] == cat['id']][0]
            seg = right_part['segmentation'][0]
            poly = np.array(seg).reshape((int(len(seg) / 2), 2))
            p = [(r[0], r[1]) for r in poly]
            right_mask = Image.new('L', (4096, 3000), 0)
            ImageDraw.Draw(right_mask).polygon(p, outline=1, fill=1)
            right_mask = np.array(right_mask)
            x, y = np.nonzero(right_mask)
            right_centroid = [np.mean(x), np.mean(y)]

            # calculate disparity
            disparities = np.abs(left_centroid[1] - right_centroid[1])
            depth = depth_from_disp(disparities)
            world_coordinates[str(cat['id'])] = convert_to_world_point(left_centroid[0], left_centroid[1], depth)

        # now calculate the pairwise distances
        for pair in dataset.keys():
            cat0, cat1 = pair[0], pair[1]
            dist = np.linalg.norm(world_coordinates[cat0] - world_coordinates[cat1])
            dataset[pair].append(dist)
        
        # TODO (@Thomas) probably no reasons to use pandas here
        df = pd.DataFrame(dataset)
        for col in df.columns.tolist():
            df[col] = df[col] / iqrs[col]
        
        pidx = np.indices((df.shape[1], df.shape[1])).reshape(2, -1)
        lcol = pd.MultiIndex.from_product([df.columns, df.columns],  names=[df.columns.name, df.columns.name])
        X = pd.DataFrame(df.values[:, pidx[0]] * df.values[:, pidx[1]],  columns=lcol)

        # load pca components
        newX = np.dot(X, components.T)
        weight = float(np.squeeze(model.predict(newX)))
        return weight

        

In [ ]:
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
stereo_frame_pairs = {}
for image in images:
    local_path = image['local_path']
    sfp_id = int(local_path.split('/')[-3])
    side = 'left' if 'left' in local_path else 'right'
    if sfp_id not in list(stereo_frame_pairs.keys()):
        stereo_frame_pairs[sfp_id] = {}
    stereo_frame_pairs[sfp_id][side] = image['id']
    
stereo_frame_pairs = list(stereo_frame_pairs.values())
    
    

In [ ]:
model = sm.load(MODEL_PATH)
components = np.load(COMPONENTS_PATH)
iqrs = pickle.load(open(IQRS_PATH, 'rb'))

output_directory = './biomass_service_output'

for i, stereo_frame_pair in enumerate(stereo_frame_pairs):
    left_image_id = stereo_frame_pair['left']
    left_image = coco.loadImgs([left_image_id])[0]
    left_image_path = left_image['local_path']
    left_annotation_ids = coco.getAnnIds(imgIds=[left_image_id])
    left_annotations = coco.loadAnns(left_annotation_ids)
    
    right_image_id = stereo_frame_pair['right']
    right_image = coco.loadImgs([right_image_id])[0]
    right_image_path = right_image['local_path']
    right_annotation_ids = coco.getAnnIds(imgIds=[right_image_id])
    right_annotations = coco.loadAnns(right_annotation_ids)
    required_categories = [cat for cat in CATEGORIES if cat['name'] in BODY_PARTS_REQUIRED]
    
    
    left_fish_detections = get_fish_detections(left_annotations, CATEGORIES, whole_fish=WHOLE_FISH)
    right_fish_detections = get_fish_detections(right_annotations, CATEGORIES, whole_fish=WHOLE_FISH)
    
    stereo_fish_detections = []
    if left_fish_detections and right_fish_detections:
        stereo_fish_detections = get_stereo_fish_detections(left_fish_detections, right_fish_detections, CATEGORIES)
        for detection in stereo_fish_detections:
            left_body_parts = detection['left']['body_parts']
            right_body_parts = detection['right']['body_parts']
            if left_body_parts and right_body_parts:
                weight = weight_estimator(left_body_parts, right_body_parts, required_categories, model, components, iqrs)
                print(weight)
                detection['biomass'] = weight
                    
    # determine which left and right fish detections were not matched
    matched_fish_detection_ids = []
    for detection in stereo_fish_detections:
        matched_fish_detection_ids.append(detection['left']['fish']['id'])
        matched_fish_detection_ids.append(detection['right']['fish']['id'])
    
    all_stereo_fish_detections = []
    for detection in left_fish_detections:
        if detection['fish']['id'] not in matched_fish_detection_ids:
            all_stereo_fish_detections.append({
                'left': detection,
                'right': None
            })
    for detection in right_fish_detections:
        if detection['fish']['id'] not in matched_fish_detection_ids:
            all_stereo_fish_detections.append({
                'left': None,
                'right': detection
            })
    
    all_stereo_fish_detections.extend(stereo_fish_detections)
    
    stereo_fish_detections_to_output = []
    left_im, right_im = Image.open(left_image_path), Image.open(right_image_path)
    for detection in all_stereo_fish_detections:
        sfd = {
            'left_crop_path': None,
            'left_annotation_json': None,
            'right_crop_path': None,
            'right_annotation_json': None,
            'biomass': None
        }
        if detection.get('left'):
            left_fish_detection = detection['left']
            
            # crop and save fish detection
            min_x, min_y, max_x, max_y = left_fish_detection['fish']['bbox']
            left_crop_basename = 'image_{}_{}_{}_{}.jpg'.format(min_x, min_y, max_x, max_y)
            left_crop_path = os.path.join(output_directory, left_crop_basename)
            left_crop = left_im.crop((min_y, min_x, max_y, max_x))
            
            left_crop.save(left_crop_path)
            
            sfd['left_crop_path'] = left_crop_path
            sfd['left_annotation_json'] = left_fish_detection
                        
        if detection.get('right'):
            right_fish_detection = detection['right']
            
            # crop and save fish detection
            min_x, min_y, max_x, max_y = right_fish_detection['fish']['bbox']
            right_crop_basename = 'image_{}_{}_{}_{}.jpg'.format(min_x, min_y, max_x, max_y)
            right_crop_path = os.path.join(output_directory, right_crop_basename)
            right_crop = right_im.crop((min_y, min_x, max_y, max_x))
            right_crop.save(right_crop_path)
            
            sfd['right_crop_path'] = right_crop_path
            sfd['right_annotation_json'] = right_fish_detection
        
        sfd['biomass'] = detection.get('biomass')
        stereo_fish_detections_to_output.append(sfd)
        print(len(stereo_fish_detections_to_output))
    
    
        
        
        
            

            

            
            
            
            
        
    
    
    

In [ ]:
plt.hist(np.squeeze(weights))
plt.xlabel('Biomass (grams)')
plt.ylabel('Count')

In [ ]:
stereo_fish_detections_to_output

In [ ]:
if not x.get('b'):
    print('hi')